In [1]:
import os
import uuid
import pandas as pd

def show_df(df):
    print(df.shape)
    display(df.head(2))

## Modèle de données pour les ingrédients

| Colonne     | Type | Description | Valeurs possibles |
|-------------|:----:|-------------|-------------------|
| id          | uuid | identifiant unique de l'ingrédient | - |
| nom         | str  | nom de l'ingrédient | - |
| type        | str  | type d'ingrédient | - |
| nutri       | dict | informations nutritionelles unitaires | - |

## Modèle de données pour les plats

| Colonne     | Type | Description | Valeurs possibles |
|-------------|:----:|-------------|-------------------|
| id          | uuid | identifiant unique du plat | - |
| nom         | str  | nom du plat | - |
| type        | str  | type de plat | ["Entrée", "Plat", "Dessert"] |
| saison      | str  | saison | ["Printemps", "Été", "Automne", "Hiver"] |
| regime      | int  | restriction en terme de régime | {0 : "Aucun", 1 : "Pescétarien", 2 : "Végétarien", 3 : "Végan"} |
| tags        | list | liste des restrictions | ["sans gluten", "sans produits laitiers", "sans sucre", "sans noix", "sans oeufs", "sans soja", "faible en FODMAP"] |
| ingredients | dict | liste d'ingrédients et quantités | - |
| recette     | str  | recette | - |
| nutri       | dict | informations nutritionelles globales | - |

## Modèle de données pour un profil

| Colonne      | Type     | Description |
|--------------|:--------:|-------------|
| id           | uuid     | identifiant unique du compte |
| username     | str      | username |
| mail         | str      | adresse mail |
| password     | str      | mot de passe |
| nom          | str      | nom |
| prenom       | str      | prénom |
| birthday     | datetime | date de naissance |
| regime       | str      | Aucun/Végan/Végétarien... |
| restrictions | list     | Liste des restrictions alimentaires (sans gluten, sans soja...) |
| goal         | str      | But de l'utilisateur (perte/gain de poids, muscu...) |

In [2]:
DATA_PATH = "/home/camille/code/Camille9999/EPSI/workshop/workshop-m1/data"

In [10]:
class NutriTable():

    def __init__(self, filename: str, columns: list) -> None:
        self.filepath = os.path.join(DATA_PATH, filename)
        if not os.path.exists(self.filepath):
            df = pd.DataFrame(columns=columns)
            df.to_csv(self.filepath, sep=";", index=False)
        self.df = pd.read_csv(self.filepath, sep=';', index_col=False)

    def add_entry(self, entries: dict | list[dict]) -> None:
        if isinstance(entries, dict):
            entries = [entries]
        for entry in entries:
            if entry.get("nom") not in self.df.nom.unique():
                if all(key in self.df.columns for key in entry.keys()):
                    entry["id"] = str(uuid.uuid4())
                    self.df.loc[len(self.df)] = entry
                    self.df.to_csv(self.filepath, sep=";", index=False)
                else:
                    unknown_cols = [col for col in entry.keys() if col not in self.df.columns]
                    raise ValueError(f"Unknown columns: {unknown_cols}")
            else:
                print(f"Warning: There is already an entry with the name `{entry.get('nom')}`")

    def mod_entry(self, id: str, entries: dict | list[dict]) -> None:
        if isinstance(entries, dict):
            entries = [entries]
        for values in entries:
            if id in self.df['id'].values:
                for key, value in values.items():
                    if key in self.df.columns and value is not None:
                        self.df.loc[self.df['id'] == id, key] = value
                self.df.to_csv(self.filepath, sep=";", index=False)
            else:
                print(f"Warning: ID `{id}` doesn't exist")

    def del_entry(self, id: str) -> None:
        if id in self.df['id'].values:
            self.df = self.df[self.df['id'] != id]
            self.df.to_csv(self.filepath, sep=";", index=False)
        else:
            print(f"Warning: ID `{id}` doesn't exist")

    def get_id_from_name(self, name: str) -> str:
        if name in self.df['nom'].values:
            return self.df.loc[self.df['nom'] == name, 'id'].values[0]
        else:
            raise ValueError(f"Name `{name}` doesn't exist")

    def get_df(self) -> pd.DataFrame:
        return self.df


In [11]:
class Ingredients(NutriTable):

    def __init__(self) -> None:
        columns = ['id', 'nom', 'type', 'nutri']
        super().__init__("ingredients.csv", columns)


class Plats(NutriTable):

    def __init__(self) -> None:
        columns = ['id', 'nom', 'type', 'saison', 'regime', 'tags', 'ingredients', 'recette', 'nutri']
        super().__init__("plats.csv", columns)


In [12]:
ingredients = Ingredients()
ingredients.get_df()

,id,nom,type,nutri


In [13]:
liste_ingredients = [
    {"nom" : "tomate", "type": "légume"},
    {"nom" : "mozzarella", "type": "fromage"},
    {"nom" : "huile d'olive", "type" : "huile végétale"},
    {"nom" : "poulet", "type": "viande"},
    {"nom" : "pomme de terre", "type" : "légume"},
    {"nom" : "poivron", "type" : "légume"},
    {"nom" : "herbes de provence", "type": "herbe"},
    {"nom" : "cerise", "type" : "fruit"},
    {"nom" : "farine", "type" : "blé"},
    {"nom" : "sucre", "type" : "sucre"},
    {"nom" : "oeuf", "type" : "oeuf"},
    {"nom" : "beurre", "type" : "produit laitier"},
    {"nom" : "comcombre", "type" : "légume"},
    {"nom" : "menthe", "type" : "herbe"},
    {"nom" : "riz", "type" : "riz"},
    {"nom" : "asperge", "type" : "légume"},
    {"nom" : "champignons de Paris", "type" : "champignon"},
    {"nom" : "mangue", "type": "fruit"},
    {"nom" : "quinoa", "type": "céréale"},
    {"nom" : "avocat", "type": "fruit"},
    {"nom" : "lait de coco", "type": "lait végétal"},
    {"nom" : "pêche", "type": "fruit"},
    {"nom" : "framboise", "type": "fruit"},
    {"nom" : "melon", "type": "fruit"},
    {"nom" : "jambon cru", "type": "viande"},
    {"nom" : "poisson", "type": "poisson"},
    {"nom" : "betterave", "type": "légume"},
    {"nom" : "orange", "type": "fruit"},
    {"nom" : "citron", "type": "fruit"},
    {"nom" : "carotte", "type": "légume"},
    {"nom" : "raisin sec", "type": "fruit"},
    {"nom" : "vanille", "type": "épice"},
    {"nom" : "courgette", "type": "légume"},
    {"nom" : "basilic", "type": "herbe"},
    {"nom" : "tofu", "type": "protéine végétale"},
    {"nom" : "aneth", "type": "herbe"}
]

In [14]:
ingredients.add_entry(liste_ingredients)
ingredients.get_df()

,id,nom,type,nutri
0,c091c7e8-3731-445f-9b17-8ccf6922285a,tomate,légume,NaN
1,4d025e8f-82d7-48ca-a673-2f6da1549f86,mozzarella,fromage,NaN
2,f077aa0f-a35b-4cf3-8536-4a11b611fcf5,huile d'olive,huile végétale,NaN
3,6b55aafa-91c6-4b34-acc7-cdeb46412cfc,poulet,viande,NaN
4,00b3ebe5-9e40-47a0-a6e8-19fff0fa4428,pomme de terre,légume,NaN
5,faf2bbeb-8f3b-45c2-bed9-ef1640b844d1,poivron,légume,NaN
6,9bb51129-d68a-4a1d-8535-5b68ef94a51e,herbes de provence,herbe,NaN
7,f32cd746-65d9-4d80-99bc-56faa2abde20,cerise,fruit,NaN
8,1a240a91-a8fc-4c5d-8008-857fcafbcfe0,farine,blé,NaN
9,22d93a12-cb82-40fb-ac9b-1ac35809fe59,sucre,sucre,NaN


In [15]:
plats = Plats()
plats.get_df()


,id,nom,type,saison,regime,tags,ingredients,recette,nutri


In [17]:
liste_plats = [
    {
        "nom": "Salade de tomates et mozzarella",
        "type": "Entrée",
        "saison": "Été",
        "regime": 2,
        "tags": ["produit laitier"],
        "ingredients": ["tomate", "mozzarella", "huile d'olive"],
        "recette": "Couper les tomates et la mozza"
    },
    {
        "nom": "Poulet grillé aux herbes et légumes de saison",
        "type": "Plat",
        "saison": "Été",
        "regime": 0,
        "tags": [],
        "ingredients": ["poulet", "herbes de provence", "pomme de terre", "poivron"],
        "recette": "Griller le poulet et les légumes"
    },
    {
        "nom": "Tarte aux cerises",
        "type": "Dessert",
        "saison": "Été",
        "regime": 2,
        "tags": ["gluten", "produit laitier"],
        "ingredients": ["cerise", "farine", "sucre", "oeuf", "beurre"],
        "recette": "Cuire la tarte aux cerises"
    },
    {
        "nom": "Gaspacho de concombre et menthe",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["concombre", "menthe"],
        "recette": "Mixer concombre et menthe"
    },
    {
        "nom": "Risotto aux asperges et champignons",
        "type": "Plat",
        "saison": "Été",
        "regime": 3,
        "tags": ["gluten"],
        "ingredients": ["riz", "asperge", "champignons de Paris"],
        "recette": "Cuire le risotto"
    },
    {
        "nom": "Sorbet à la mangue",
        "type": "Dessert",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["mangue"],
        "recette": "Congeler la mangue mixée"
    },
    {
        "nom": "Salade de quinoa, avocat et mangue",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["quinoa", "avocat", "mangue"],
        "recette": "Mélanger quinoa, avocat et mangue"
    },
    {
        "nom": "Curry de légumes au lait de coco",
        "type": "Plat",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["légumes de saison", "lait de coco"],
        "recette": "Cuire les légumes au curry"
    },
    {
        "nom": "Compote de pêches et framboises",
        "type": "Dessert",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["pêche", "framboise"],
        "recette": "Cuire les fruits"
    },
    {
        "nom": "Salade de melon et jambon cru",
        "type": "Entrée",
        "saison": "Été",
        "regime": 0,
        "tags": [],
        "ingredients": ["melon", "jambon cru"],
        "recette": "Mélanger melon et jambon"
    },
    {
        "nom": "Filet de poisson grillé et légumes rôtis",
        "type": "Plat",
        "saison": "Été",
        "regime": 1,
        "tags": [],
        "ingredients": ["poisson", "légumes de saison"],
        "recette": "Griller le poisson et les légumes"
    },
    {
        "nom": "Salade de betteraves et oranges",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["betterave", "orange"],
        "recette": "Mélanger betterave et orange"
    },
    {
        "nom": "Poulet au citron et riz basmati",
        "type": "Plat",
        "saison": "Été",
        "regime": 0,
        "tags": [],
        "ingredients": ["poulet", "citron", "riz"],
        "recette": "Cuire le poulet au citron"
    },
    {
        "nom": "Sorbet à la fraise",
        "type": "Dessert",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["fraise"],
        "recette": "Congeler la fraise mixée"
    },
    {
        "nom": "Salade de carottes râpées et raisins secs",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["carotte", "raisin sec"],
        "recette": "Mélanger carottes et raisins"
    },
    {
        "nom": "Ratatouille et riz complet",
        "type": "Plat",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["légumes de saison", "riz complet"],
        "recette": "Cuire la ratatouille"
    },
    {
        "nom": "Panna cotta à la vanille",
        "type": "Dessert",
        "saison": "Été",
        "regime": 2,
        "tags": ["produit laitier"],
        "ingredients": ["vanille", "lait", "sucre"],
        "recette": "Cuire la panna cotta"
    },
    {
        "nom": "Soupe froide de courgettes et basilic",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["courgette", "basilic"],
        "recette": "Mixer courgette et basilic"
    },
    {
        "nom": "Tofu mariné et légumes sautés",
        "type": "Plat",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["tofu", "légumes de saison"],
        "recette": "Sauter tofu et légumes"
    },
    {
        "nom": "Salade de concombre et aneth",
        "type": "Entrée",
        "saison": "Été",
        "regime": 3,
        "tags": [],
        "ingredients": ["concombre", "aneth"],
        "recette": "Mélanger concombre et aneth"
    },
    {
        "nom": "Poulet grillé et courgettes rôties",
        "type": "Plat",
        "saison": "Été",
        "regime": 0,
        "tags": [],
        "ingredients": ["poulet", "courgette"],
        "recette": "Griller poulet et courgettes"
    }
]


In [18]:
plats.add_entry(liste_plats)
plats.get_df()

,id,nom,type,saison,regime,tags,ingredients,recette,nutri
0,58d738ee-1428-4730-a86c-d91801bf7dc7,Salade de tomates et mozzarella,Entrée,Été,2,[produit laitier],"[tomate, mozzarella, huile d'olive]",Couper les tomates et la mozza,NaN
1,62e16a12-f911-4312-8788-d4d37daef1f6,Poulet grillé aux herbes et légumes de saison,Plat,Été,0,[],"[poulet, herbes de provence, pomme de terre, p...",Griller le poulet et les légumes,NaN
2,2b6fe3fc-2901-4300-b1fb-0572d7757fc5,Tarte aux cerises,Dessert,Été,2,"[gluten, produit laitier]","[cerise, farine, sucre, oeuf, beurre]",Cuire la tarte aux cerises,NaN
3,c3affb6e-4ef9-402f-810d-54b75cef4e79,Gaspacho de concombre et menthe,Entrée,Été,3,[],"[concombre, menthe]",Mixer concombre et menthe,NaN
4,5b6ef890-3866-4ad4-b083-e3b26d7a3864,Risotto aux asperges et champignons,Plat,Été,3,[gluten],"[riz, asperge, champignons de Paris]",Cuire le risotto,NaN
5,76bfa803-20ba-4361-9215-53c582e94187,Sorbet à la mangue,Dessert,Été,3,[],[mangue],Congeler la mangue mixée,NaN
6,e7e050c9-d3fc-453d-bfe0-9ae53b04f442,"Salade de quinoa, avocat et mangue",Entrée,Été,3,[],"[quinoa, avocat, mangue]","Mélanger quinoa, avocat et mangue",NaN
7,79e48b20-52a6-4d91-8dc1-a8acbbb04394,Curry de légumes au lait de coco,Plat,Été,3,[],"[légumes de saison, lait de coco]",Cuire les légumes au curry,NaN
8,b4a6c3e9-db12-4372-9465-4de34cc5fbaf,Compote de pêches et framboises,Dessert,Été,3,[],"[pêche, framboise]",Cuire les fruits,NaN
9,1cdf2e73-7f6b-4beb-8bee-965292d59268,Salade de melon et jambon cru,Entrée,Été,0,[],"[melon, jambon cru]",Mélanger melon et jambon,NaN


In [95]:
liste_ingredients = ["tomate", "mozzarella"]

plats.add_entry({
    "nom" : "Salade de tomates et mozzarella",
    "type" : "Entrée",
    "saison" : "Été",
    "regime" : 2,
    "tags" : "dairy",
    "ingredients" : {
        ingredients.get_id_from_name(ingredient) : {
            "nom" : ingredient,
            "quantite" : 1
        } for ingredient in liste_ingredients
    },
    "recette" : "Couper les tomates et la mozza",
})
plats.get_df()

,id,nom,type,saison,regime,tags,ingredients,recette,nutri
0,891c18e5-5bbd-46d0-b411-e15b575cf954,Salade de tomates et mozzarella,Entrée,Été,2,dairy,{'879cc95a-a1f9-424c-b12a-2bfd4e07537c': {'nom...,Couper les tomates et la mozza,NaN


In [8]:
ingredients.add_entry({"nom" : "poulet", "type" : "volaille", "nutri" : None})
ingredients.add_entry({"nom" : "pomme de terre", "type" : "légume", "nutri" : None})
ingredients.add_entry({"nom" : "courgette", "type" : "légume", "nutri" : None})
ingredients.add_entry({"nom" : "poivron", "type" : "légume", "nutri" : None})
ingredients.add_entry({"nom" : "herbes de provence", "type" : "herbe", "nutri" : None})
ingredients.get_df()

,id,nom,type,nutri
0,879cc95a-a1f9-424c-b12a-2bfd4e07537c,tomate,fruit,NaN
1,9acbc867-b3d9-4c64-9562-ca87a3982266,mozzarella,fromage,NaN
2,6ade4ffd-c9d8-47df-8611-8a99c39d5a9c,huile d'olive,huile végétale,NaN
3,cd4a9927-6cf0-48d0-9095-e949a50ccea8,poulet,volaille,NaN
4,7f61d010-7e12-44d6-8826-68a5bbc9e7d7,pomme de terre,légume,NaN
5,fe670b8d-b85a-4e81-8579-c7f4266cb61a,courgette,légume,NaN
6,833c0c0c-cba6-4d21-9f7d-026e3635f6ca,poivron,légume,NaN
7,1fd6101a-80b4-4311-807d-858d0e04a869,herbes de provence,herbe,NaN


In [10]:
liste_ingredients = ["poulet", "pomme de terre", "courgette", "tomate", "poivron", "huile d'olive", "herbes de provence"]

plats.add_entry({
    "nom" : "Poulet grillé aux herbes et légumes de saison",
    "type" : "Plat",
    "saison" : "Été",
    "regime" : 2,
    "tags" : "viande",
    "ingredients" : {
        ingredients.get_id_from_name(ingredient) : {
            "nom" : ingredient,
            "quantite" : 1
        } for ingredient in liste_ingredients
    },
    "recette" : "",
})
plats.get_df()

,id,nom,type,saison,regime,tags,ingredients,recette,nutri
0,891c18e5-5bbd-46d0-b411-e15b575cf954,Salade de tomates et mozzarella,Entrée,Été,2,dairy,{'879cc95a-a1f9-424c-b12a-2bfd4e07537c': {'nom...,Couper les tomates et la mozza,NaN
1,8aeb0ee1-6a9f-4b9b-914a-99932bfc1a00,Poulet grillé aux herbes et légumes de saison,Plat,Été,2,viande,{'cd4a9927-6cf0-48d0-9095-e949a50ccea8': {'nom...,,NaN


- **Dessert** : Tarte aux cerises


### Menu 2 : Végétarien
- **Entrée** : Gaspacho de concombre et menthe
  - Tags : #Végétarien #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Plat** : Risotto aux asperges et champignons
  - Tags : #Végétarien #SansGluten #SansNoix #SansŒufs
- **Dessert** : Sorbet à la mangue
  - Tags : #Végétarien #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP

### Menu 3 : Végan
- **Entrée** : Salade de quinoa, avocat et mangue
  - Tags : #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Plat** : Curry de légumes au lait de coco
  - Tags : #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja
- **Dessert** : Compote de pêches et framboises
  - Tags : #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP

### Menu 4 : Sans gluten
- **Entrée** : Salade de melon et jambon cru
  - Tags : #SansGluten #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Plat** : Filet de poisson grillé et légumes rôtis
  - Tags : #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Dessert** : Salade de fruits frais
  - Tags : #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP

### Menu 5 : Sans produits laitiers
- **Entrée** : Salade de betteraves et oranges
  - Tags : #SansProduitsLaitiers #SansGluten #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Plat** : Poulet au citron et riz basmati
  - Tags : #SansProduitsLaitiers #SansGluten #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP
- **Dessert** : Sorbet à la fraise
  - Tags : #SansProduitsLaitiers #Végan #SansGluten #SansNoix #SansŒufs #SansSoja #FaibleEnFODMAP

### Menu 6 : Sans noix
- **Entrée** : Salade de carottes râpées et raisins secs
  - Tags : #SansNoix #Végan #SansGluten #SansProduitsLaitiers #SansŒufs #SansSoja #FaibleEnFODMAP
- **Plat** : Ratatouille et riz complet
  - Tags : #SansNoix #Végan #SansGluten #SansProduitsLaitiers #SansŒufs #SansSoja
- **Dessert** : Panna cotta à la vanille (sans noix)
  - Tags : #SansNoix #SansGluten #SansŒufs #SansSoja

### Menu 7 : Sans œufs
- **Entrée** : Soupe froide de courgettes et basilic
  - Tags : #SansŒufs #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansSoja #FaibleEnFODMAP
- **Plat** : Tofu mariné et légumes sautés
  - Tags : #SansŒufs #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansSoja
- **Dessert** : Salade de fruits exotiques
  - Tags : #SansŒufs #Végan #SansGluten #SansProduitsLaitiers #SansNoix #SansSoja #FaibleEnFODMAP

### Menu 8 : Faible en FODMAP
- **Entrée** : Salade de concombre et aneth
  - Tags : #FaibleEnFODMAP #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja
- **Plat** : Poulet grillé et courgettes rôties
  - Tags : #FaibleEnFODMAP #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja
- **Dessert** : Gelée de fruits rouges
  - Tags : #FaibleEnFODMAP #SansGluten #SansProduitsLaitiers #SansNoix #SansŒufs #SansSoja